### Data Loading

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving diabetes.csv to diabetes.csv


In [ ]:
import collections
import sys
import os
import io
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
import logging
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
logging.getLogger("tensorflow").setLevel(logging.WARNING)

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['diabetes.csv']))
SEED = 42
np.random.seed(SEED)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


### Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

1. Tune Batch Size and Number of Epochs

In [ ]:
def CreateModel():
  model = Sequential()
  model.add(Dense(12, input_dim=8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  return model

In [ ]:
model = KerasClassifier(build_fn=CreateModel, verbose=0)
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
best_score = grid_results.best_score_
best_params = grid_results.best_params_

print(f'Best Score: {best_score}')
print(f'Best Parametes: {best_params}')

Best Score: 0.6979166666666666
Best Parametes: {'batch_size': 10, 'epochs': 50}


2. Tune Training Optimization Algorithm

In [ ]:
def CreateModel(optimizer='adam'):
  model = Sequential()
  model.add(Dense(12, input_dim=8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  
  return model

In [ ]:
model = KerasClassifier(build_fn=CreateModel, batch_size=10, epochs=100, verbose=0)
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
best_score = grid_results.best_score_
best_params = grid_results.best_params_

print(f'Best Score: {best_score}')
print(f'Best Parametes: {best_params}')

Best Score: 0.703125
Best Parametes: {'optimizer': 'Nadam'}


3. Tune Learning Rate and Momentum




In [ ]:
from keras.optimizers import SGD

def CreateModel(learn_rate=0.01, momentum=0):
  model = Sequential()
  model.add(Dense(12, input_dim=8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = SGD(lr=learn_rate, momentum=momentum)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  
  return model

In [ ]:
model = KerasClassifier(build_fn=CreateModel, batch_size=10, epochs=100, verbose=0)
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
best_score = grid_results.best_score_
best_params = grid_results.best_params_

print(f'Best Score: {best_score}')
print(f'Best Parametes: {best_params}')

Best Score: 0.6848958333333334
Best Parametes: {'learn_rate': 0.001, 'momentum': 0.4}


4. Tune Network Weight Initialization


In [ ]:
def CreateModel(init_mode='uniform'):
  model = Sequential()
  model.add(Dense(12, kernel_initializer=init_mode, input_dim=8, activation='relu'))
  model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
  
  return model

In [ ]:
model = KerasClassifier(build_fn=CreateModel, batch_size=10, epochs=100, verbose=0)
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
best_score = grid_results.best_score_
best_params = grid_results.best_params_

print(f'Best Score: {best_score}')
print(f'Best Parametes: {best_params}')

Best Score: 0.7239583333333334
Best Parametes: {'init_mode': 'lecun_uniform'}


5. Tune Neuron Activation Function

In [ ]:
def CreateModel(activation='relu'):
  model = Sequential()
  model.add(Dense(12, kernel_initializer='lecun_uniform', input_dim=8, activation=activation))
  model.add(Dense(1, kernel_initializer='lecun_uniform', activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
  
  return model

In [ ]:
model = KerasClassifier(build_fn=CreateModel, batch_size=10, epochs=100, verbose=0)
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
best_score = grid_results.best_score_
best_params = grid_results.best_params_

print(f'Best Score: {best_score}')
print(f'Best Parametes: {best_params}')

Best Score: 0.7018229166666666
Best Parametes: {'activation': 'softplus'}


6. Tune Dropout Regularization

In [ ]:
from keras.constraints import maxnorm
from keras.layers import Dropout

def CreateModel(dropout_rate=0.0, weight_constraint=0):
  model = Sequential()
  model.add(Dense(12, kernel_initializer='lecun_uniform', input_dim=8, activation='softplus', kernel_constraint=maxnorm(weight_constraint)))
  model.add(Dropout(dropout_rate))
  model.add(Dense(1, kernel_initializer='lecun_uniform', activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
  
  return model

In [ ]:
model = KerasClassifier(build_fn=CreateModel, batch_size=10, epochs=100, verbose=0)
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

param_grid = dict(weight_constraint=weight_constraint, dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
best_score = grid_results.best_score_
best_params = grid_results.best_params_

print(f'Best Score: {best_score}')
print(f'Best Parametes: {best_params}')

Best Score: 0.7109375
Best Parametes: {'dropout_rate': 0.1, 'weight_constraint': 1}


7. Tune Number of Neurons in the Hidden Layer

In [ ]:
from keras.constraints import maxnorm
from keras.layers import Dropout

def CreateModel(neurons=1):
  model = Sequential()
  model.add(Dense(neurons, kernel_initializer='lecun_uniform', input_dim=8, activation='softplus', kernel_constraint=maxnorm(1)))
  model.add(Dropout(0.1))
  model.add(Dense(1, kernel_initializer='lecun_uniform', activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
  
  return model

In [ ]:
model = KerasClassifier(build_fn=CreateModel, batch_size=10, epochs=100, verbose=0)
neurons = [1, 5, 10, 15, 20, 25, 30]

param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_results = grid.fit(X, y)
best_score = grid_results.best_score_
best_params = grid_results.best_params_

print(f'Best Score: {best_score}')
print(f'Best Parametes: {best_params}')

Best Score: 0.7109375
Best Parametes: {'neurons': 25}
